**MNIST Notebook by Alan Choon Zhen Yu**

In [1]:
import pandas as pd
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(1)

In [2]:
train = pd.read_csv("../input/train.csv")

In [3]:
Y_train = train["label"]
X_train = train.drop(labels='label',axis=1)

In [4]:
X_train.head()

In [5]:
Y_train.head()

In [6]:
y_r=pd.DataFrame(Y_train.values.reshape(Y_train.shape[0],1))

**One Hot Encoding:**

In [7]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
#Change the column to a 2-dim array
y_r = enc.fit_transform(y_r)

In [8]:
pd.DataFrame(y_r).head()

**Split Dataset into training and development sets:**

In [37]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, y_r, test_size=0.20, random_state=101)

In [97]:
x_train_nn = x_train.values
x_test_nn = x_test.values

In [19]:
x_train_nn.shape

In [98]:
x_train_nn = np.reshape(x_train_nn, (33600,28,28,1))
x_train_nn.shape

In [21]:
x_test_nn.shape

In [99]:
x_test_nn = np.reshape(x_test_nn, (8400,28,28,1))
x_test_nn.shape

In [196]:
y_train.shape

In [24]:
y_test.shape

**Convolutional Neural Network:**

The model below takes inspiration from Convolutional Neural Networks course taught by deeplearning.ai

 as well as from: https://www.tensorflow.org/tutorials/layers

In [100]:
x_train_nn = x_train_nn/255
x_test_nn = x_test_nn/255

In [26]:
import tensorflow as tf
from tensorflow.python.framework import ops

**Create Placeholders:**

In [27]:
def create_placeholders(n_H0, n_W0,n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0,n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    ### START CODE HERE ### (≈2 lines)
    X = tf.placeholder("float",(None,n_H0,n_W0,n_C0),name="X")
    Y = tf.placeholder("float",(None,n_y),"Y")
    ### END CODE HERE ###
    
    return X, Y

In [19]:
#Test case:
X, Y = create_placeholders(64, 64, 1, 10)
print ("X = " + str(X))
print ("Y = " + str(Y))

**Initialize Parameters:**

In [93]:
def initialize_parameters():
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [5, 5, 1, 32]
                        W2 : [5, 5, 32, 64]
                        
    where shape: [n_H, n_W, n_C_prev, n_C]

    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)                              # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 2 lines of code)
    W1 = tf.get_variable("W1",[5,5,1,32],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    b1 = tf.get_variable("b1",[1,1,1,32],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2",[5,5,32,64],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    b2 = tf.get_variable("b2",[1,1,1,64],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "W2": W2,
                  "b1": b1,
                  "b2": b2}
    
    return parameters

**Forward Propagation:**

Implements CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED.

In [102]:
def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    ### START CODE HERE ###
    # CONV2D: stride of 1, padding 'SAME'
    Z1 = tf.nn.conv2d(X,W1, strides = [1,1,1,1], padding = 'SAME')
    Z1 += b1
    # RELU
    A1 = tf.nn.relu(Z1)
    # MAXPOOL: window 2x2, stride 2, padding 'SAME'
    P1 = tf.nn.max_pool(A1,ksize=[1,2,2,1],strides=[1,2,2,1],padding= 'SAME')
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME')
    Z2 += b2
    # RELU
    A2 = tf.nn.relu(Z2)
    # MAXPOOL: window 2x2, stride 2, padding 'SAME'
    P2 = tf.nn.max_pool(A2,ksize=[1,2,2,1],strides = [1,2,2,1],padding='SAME')
    # FLATTEN
    P2 = tf.contrib.layers.flatten(P2)
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 10 neurons in output layer. Hint: one of the arguments should be "activation_fn=None"
    Z3 = tf.contrib.layers.fully_connected(P2,1024,activation_fn=tf.nn.relu)
    Z3 = tf.contrib.layers.dropout(Z3,keep_prob=0.5)
    Z3 = tf.contrib.layers.fully_connected(Z3,10,activation_fn=tf.nn.softmax)
    ### END CODE HERE ###

    return Z3

**Compute Cost:**

In [30]:
def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (10, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3,labels=Y))
    ### END CODE HERE ###
    
    return cost

In [31]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (number of examples, input size)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (number of examples, 1)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[0]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :]
    shuffled_Y = Y[permutation, :]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[k*mini_batch_size : k*mini_batch_size + mini_batch_size, : ]
        mini_batch_Y = shuffled_Y[k*mini_batch_size : k*mini_batch_size + mini_batch_size, : ]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[num_complete_minibatches*mini_batch_size :,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches*mini_batch_size :,:]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

**Model:**

In [203]:
def model(X_train, Y_train, X_test, Y_test, *test_set, test = False, learning_rate = 0.005,
          num_epochs = 60, minibatch_size = 2048, print_cost = True):
    """
    Implements a three-layer ConvNet in Tensorflow:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X_train -- training set, of shape (None, 28, 28, 1)
    Y_train -- test set, of shape (None, n_y = 10)
    X_test -- training set, of shape (None, 28, 28, 1)
    Y_test -- test set, of shape (None, n_y = 10)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    train_accuracy -- real number, accuracy on the train set (X_train)
    test_accuracy -- real number, testing accuracy on the test set (X_test)
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []                                        # To keep track of the cost
    
    test_set = np.reshape(test_set,(-1,28,28,1))
    # Create Placeholders of the correct shape
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters()
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = forward_propagation(X,parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z3,Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
     
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)        
        
        # Do the training loop
        for epoch in range(num_epochs):
            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , temp_cost = sess.run([optimizer,cost],feed_dict = {X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                minibatch_cost += temp_cost / num_minibatches
                

            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
            
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        
        # Calculate the correct predictions
        predict_op = tf.argmax(Z3, 1)

        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        preds = predict_op.eval(feed_dict={X: test_set})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
                
        if test == False:
            return train_accuracy, test_accuracy, parameters
        else:
            return train_accuracy, test_accuracy, parameters, preds

In [197]:
test = pd.read_csv("../input/test.csv")

In [198]:
test.values.shape

In [199]:
x_testset = np.reshape(test.values, (28000,28,28,1))

In [200]:
x_testset = x_testset/255

In [201]:
x_testset.shape

In [204]:
_, _, parameters, preds = model(x_train_nn, y_train, x_test_nn, y_test,x_testset,test=True)

With CNN, the development set accuracy has increased to a staggering ~98.6%!

In [205]:
preds

In [189]:
preds = pd.DataFrame(preds,columns=['Label'])

In [191]:
preds['ImageId'] = range(1,28001,1)

In [192]:
preds = preds[['ImageId','Label']]
preds.to_csv("preds.csv", index=False)